In [1]:
import numpy as np
import pandas as pd

import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
df = pd.read_excel('/Users/didyvanbelkom/Documents/02 MSc DSS/THESIS CODE/Jupiter Notebook/online_retail_II.xlsx', sheet_name='Year 2009-2010')

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [4]:
'The dimensions of the dataframe are:'
df.shape

(525461, 8)

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,525461.00000,10.33767,107.42411,-9600.00000,1.00000,3.00000,10.00000,19152.00000
Price,525461.00000,4.68883,146.12691,-53594.36000,1.25000,2.10000,4.21000,25111.09000
Customer ID,417534.00000,15360.64548,1680.81132,12346.00000,13983.00000,15311.00000,16799.00000,18287.00000


In [6]:
'Creating a new variable called TotalPrice to find the total income per invoice'

df["TotalPrice"] = df["Quantity"] * df["Price"]

In [7]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


In [8]:
'Check for missing observations'
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
TotalPrice          0
dtype: int64

In [9]:
df.dropna(inplace=True) #erases incomplete observations.

df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[(df['Quantity'] > 0)]
df = df[(df['Price'] > 0)]

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,407664.00000,13.58559,96.84075,1.00000,2.00000,5.00000,12.00000,19152.00000
Price,407664.00000,3.29444,34.75796,0.00100,1.25000,1.95000,3.75000,10953.50000
Customer ID,407664.00000,15368.59260,1679.76214,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000
TotalPrice,407664.00000,21.66491,77.15006,0.00100,4.95000,11.90000,19.50000,15818.40000


**Opmerking**: de negatieve waardes bij Quantity en Price zijn weggehaald dus de retouren zitten er niet meer in. Ook de missende CustomerID regels zijn eruit gehaald omdat die essentieel zijn om te gebruiken voor de RFM analyse omdat die per klantID gaat.

# Segmentation

In [11]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [12]:
df["InvoiceDate"].max()

Timestamp('2010-12-09 20:01:00')

In [13]:
today_date = dt.datetime(2011, 12, 11)


rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.00000,530,11,372.86000
12347.00000,368,2,1323.32000
12348.00000,439,1,222.16000
12349.00000,408,3,2671.14000
12351.00000,376,1,300.93000


In [14]:
rfm.columns = ['recency', 'frequency', 'monetary']

In [15]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])


rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [16]:
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

rfm.head()

rfm.describe().T

rfm[rfm["RFM_SCORE"] == "55"].head()

rfm[rfm["RFM_SCORE"] == "11"].head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12355.00000,568,1,488.21000,1,1,2,11
12362.00000,739,1,130.00000,1,1,1,11
12366.00000,634,1,500.24000,1,1,2,11
12368.00000,629,1,917.70000,1,1,3,11
12378.00000,563,1,1407.70000,1,1,4,11


**Opmerking**: Tot hier de code gebruikt van Kaggle. Daarna deelt hij hem op in segmenten maar vanaf daar gaan wij de clustering toepassen met KNN = 4 om te testen.

In [17]:
rfm.describe()

,recency,frequency,monetary
count,4312.00000,4312.00000,4312.00000
mean,456.17254,4.45571,2048.23824
std,96.86146,8.17021,8914.48128
min,366.00000,1.00000,2.95000
25%,383.00000,1.00000,307.98750
50%,418.00000,2.00000,706.02000
75%,501.00000,5.00000,1723.14250
max,739.00000,205.00000,349164.35000


# Creating binary classifier "Top 10% spender"

The goal is to predict whether a customer will be a top 10% spend customer predicted based on the RFM scores and the clusters they are in. First we import the second year data as "df_y2".

In [18]:
import pandas as pd
df_y2 = pd.read_excel('/Users/didyvanbelkom/Documents/02 MSc DSS/THESIS CODE/Jupiter Notebook/online_retail_II.xlsx', sheet_name='Year 2010-2011')
df_y2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom


For this dataframe we also remove the empty cells and the negative values for price and quantity since we want to keep the purchase history and not the returns.

In [19]:
df_y2.dropna(inplace=True) #erases incomplete observations.

df_y2 = df_y2[~df_y2["Invoice"].str.contains("C", na=False)]
df_y2 = df_y2[(df_y2['Quantity'] > 0)]
df_y2 = df_y2[(df_y2['Price'] > 0)]

df_y2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom


We need to know the total spend by each customer so we calculate the value of each row and then group by Customer ID.

In [20]:
df_y2["TotalPrice"] = df_y2["Quantity"] * df_y2["Price"]

In [21]:
df_y2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom,15.30000
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom,22.00000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000


In [22]:
test = df_y2.groupby("Customer ID")["TotalPrice"].sum().reset_index()
test.head()

,Customer ID,TotalPrice
0,12346.00000,77183.60000
1,12347.00000,4310.00000
2,12348.00000,1797.24000
3,12349.00000,1757.55000
4,12350.00000,334.40000


In [23]:
test2 = test.sort_values(by='TotalPrice',ascending=False)
test2.head(10)

,Customer ID,TotalPrice
1689,14646.00000,280206.02000
4201,18102.00000,259657.30000
3728,17450.00000,194550.79000
3008,16446.00000,168472.50000
1879,14911.00000,143825.06000
55,12415.00000,124914.53000
1333,14156.00000,117379.63000
3771,17511.00000,91062.38000
2702,16029.00000,81024.84000
0,12346.00000,77183.60000


In [24]:
test2.describe()

,Customer ID,TotalPrice
count,4338.00000,4338.00000
mean,15300.40802,2054.27061
std,1721.80849,8989.22990
min,12346.00000,3.75000
25%,13813.25000,307.41500
50%,15299.50000,674.48500
75%,16778.75000,1661.74000
max,18287.00000,280206.02000


Now we create a binary variable with 0 = no and 1 = yes for whether or not a customer is in the top 10% spend category. Testing if the top values got a "Yes" as value:

In [25]:
test2["Top 10% spend"] = pd.cut(test2.TotalPrice,bins=[3.74,3649.100000,280206.020000],labels=[0,1]).astype(int)
test2.head()

,Customer ID,TotalPrice,Top 10% spend
1689,14646.00000,280206.02000,1
4201,18102.00000,259657.30000,1
3728,17450.00000,194550.79000,1
3008,16446.00000,168472.50000,1
1879,14911.00000,143825.06000,1


Cheking if the lowest values have gotten a "No" as value:

In [26]:
test2.tail()

,Customer ID,TotalPrice,Top 10% spend
3323,16878.00000,13.30000,0
4098,17956.00000,12.75000,0
3014,16454.00000,6.90000,0
1793,14792.00000,6.20000,0
3217,16738.00000,3.75000,0


Now, we create the final dataframe by combining the RFM value frame with the column 'Top 10% spend'

In [27]:
df_complete = pd.merge(left=rfm, right=test2, left_on='Customer ID', right_on='Customer ID')
df_complete.head()

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,TotalPrice,Top 10% spend
0,12346.00000,530,11,372.86000,2,5,2,25,77183.60000,1
1,12347.00000,368,2,1323.32000,5,2,4,52,4310.00000,1
2,12348.00000,439,1,222.16000,2,1,1,21,1797.24000,0
3,12349.00000,408,3,2671.14000,3,3,5,33,1757.55000,0
4,12352.00000,376,2,343.80000,5,2,2,52,2506.04000,0


In [28]:
df_complete['Top 10% spend'] = df_complete['Top 10% spend'].astype(int)

In [29]:
df_complete.describe()

,Customer ID,recency,frequency,monetary,TotalPrice,Top 10% spend
count,2772.00000,2772.00000,2772.00000,2772.00000,2772.00000,2772.00000
mean,15344.84019,427.17713,5.78896,2786.05637,2583.65320,0.13781
std,1703.41156,75.37833,9.77612,10982.59053,10552.10906,0.34476
min,12346.00000,366.00000,1.00000,13.52000,6.90000,0.00000
25%,13879.50000,377.00000,2.00000,484.83750,351.02000,0.00000
50%,15346.50000,397.00000,3.00000,1079.87500,841.24500,0.00000
75%,16826.00000,442.00000,7.00000,2396.90500,2119.87500,0.00000
max,18287.00000,739.00000,205.00000,349164.35000,280206.02000,1.00000


In [30]:
df_complete.isnull().sum()

Customer ID        0
recency            0
frequency          0
monetary           0
recency_score      0
frequency_score    0
monetary_score     0
RFM_SCORE          0
TotalPrice         0
Top 10% spend      0
dtype: int64

In [31]:
df_complete.to_excel(r'/Users/didyvanbelkom/Documents/02 MSc DSS/THESIS CODE/Jupiter Notebook/df_complete_2.xlsx', index = False)

# Clustering

In [32]:
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# import required libraries for clustering
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

df_clustering = df_complete[['recency_score', 'frequency_score', 'monetary_score']]
df_clustering.head()

,recency_score,frequency_score,monetary_score
0,2,5,2
1,5,2,4
2,2,1,1
3,3,3,5
4,5,2,2


In [34]:
# Silhouette analysis
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:
    
    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(df_clustering)
    
    cluster_labels = kmeans.labels_
    
    # silhouette score
    silhouette_avg = silhouette_score(df_clustering, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))

For n_clusters=2, the silhouette score is 0.4343719073392716
For n_clusters=3, the silhouette score is 0.3633792897137216
For n_clusters=4, the silhouette score is 0.3998906099350407
For n_clusters=5, the silhouette score is 0.3707912884025397
For n_clusters=6, the silhouette score is 0.358086684802631
For n_clusters=7, the silhouette score is 0.3720439043628
For n_clusters=8, the silhouette score is 0.3762756057828359
